In [17]:
from pprint import pprint

from selenium import webdriver
from bs4 import BeautifulSoup

In [18]:
# Create the browser

def create_browser():
    """Creates and returns a Selenium Chrome webdriver."""
    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    browser = webdriver.Chrome(
        executable_path='../../utilities/chromedriver', 
        chrome_options=options)
    return browser

def get_html(as_show_link):
    """Returns the HTML content for a given URL."""
    browser = create_browser()
    browser.get(as_show_link)
    html = browser.page_source
    browser.quit()
    return html

def normalize_season(season):
    episodes = season.find_all('div', class_='_29ThWwPi')
    season_number_tag = season.find(name='meta', attrs={'itemprop':'seasonNumber'})
    season_number = season_number_tag['content']

    episode_guide = {}

    for index, episode in enumerate(episodes):
        episode_data = {}
        episode_number_tag = episode.find('meta', attrs={'itemprop':'episodeNumber'})
        episode_number = episode_number_tag['content']
        meta_tags = episode.find_all('meta')

        for meta_data in meta_tags:
            prop = meta_data.get('itemprop')
            content = meta_data.get('content')
            if prop and content:
                episode_data[prop] = content
        episode_guide[f'episode_{episode_number}'] = episode_data

    return season_number, episode_guide

def create_show_guide(show_link):
    html = get_html(show_link)
    soup = BeautifulSoup(html, 'html.parser')
    seasons = soup.find_all(name='div', attrs={'itemprop':'containsSeason'})
    show_guide = {}

    for index, season in enumerate(seasons):
        season_number, episode_guide = normalize_season(season)
        show_guide[f'season_{season_number}'] = episode_guide

    return show_guide


In [19]:
adultswim_show_link = 'https://www.adultswim.com/videos/american-dad'
show_guide = create_show_guide(adultswim_show_link)

In [20]:
pprint(show_guide)

{'season_10': {'episode_1': {'contentRating': 'TV-14',
                             'datePublished': '2014-10-21T01:00:00.000Z',
                             'description': 'When her efforts to save the '
                                            'planet prove hopeless, Hayley '
                                            'decides that she might get more '
                                            'attention if she becomes a '
                                            'blonde. Meanwhile, Stan and Steve '
                                            'go on a mission to find a new '
                                            'home for the Smiths.',
                             'duration': 'T21M1S',
                             'episodeNumber': '1',
                             'expires': '2020-09-19T03:59:00.000Z',
                             'thumbnailUrl': 'https://i.cdn.turner.com/adultswim/big/image-upload/thumbnails/thumb-2_image-151991856208418.jpg',
                        